# 🚀 Getting Started with This Template

## Before Running This Notebook:

1. **Replace placeholder values** in the Configuration Setup section with your actual Azure credentials
2. **Install required packages**: `pip install azure-ai-projects azure-identity`
3. **Set up Azure AI Foundry project** following the prerequisites section
4. **Run cells sequentially** - each builds on the previous

## This is a Template Repository
- All credential values are placeholders
- Replace them with your actual Azure resource information
- Never commit real credentials to any repository

# Azure AI Agent Demo - Complete Educational Guide

## 🎯 Learning Objectives
By the end of this session, you will understand:
- How to create and configure Azure AI Agents
- How to implement custom functions for AI agents
- How conversation threads work in AI agent systems
- Best practices for resource management and cleanup
- Real-world scenarios and use cases for AI agents

## 📚 What are Azure AI Agents?

**Azure AI Agents** are intelligent assistants that can:
- 🧠 **Understand natural language** requests
- 🔧 **Execute custom functions** to perform tasks
- 💬 **Maintain conversation context** across multiple interactions
- 🔄 **Chain multiple operations** together automatically
- 📊 **Process and return structured data**

### Key Components:
1. **Agent** - The AI brain that processes requests
2. **Functions** - Custom code the agent can execute
3. **Threads** - Conversation sessions with memory
4. **Messages** - Individual exchanges within a thread
5. **Runs** - Execution instances of agent processing

This notebook demonstrates practical implementation with real-world scenarios.

## 🔐 Azure Setup Prerequisites

Before running this notebook, you need:

### 1. Azure AI Foundry Project
- Create an Azure AI Foundry project in Azure portal
- Note down your project endpoint URL
- Ensure you have appropriate permissions

### 2. App Registration (Service Principal)
The credentials below come from **Azure App Registration**:

#### Steps to create App Registration:
1. **Azure Portal** → **Azure Active Directory** → **App registrations**
2. **New registration** → Give it a name (e.g., "AI-Agent-Demo")
3. **Copy the following values:**
   - **Application (client) ID** → `AZURE_CLIENT_ID`
   - **Directory (tenant) ID** → `AZURE_TENANT_ID`
4. **Certificates & secrets** → **New client secret** → Copy value → `AZURE_CLIENT_SECRET`
5. **Grant permissions** to your AI Foundry project

### 3. Required Permissions
Your App Registration needs:
- **Cognitive Services User** role on the AI Foundry project
- **AI Developer** role (if available)
- **Contributor** access to the resource group (for agent creation)

### 4. Install Required Packages
```bash
pip install azure-ai-projects azure-identity
```

## 📦 Install Required Packages

Install the Azure AI SDK packages needed for this demo:

In [ ]:
# Uncomment and run if packages aren't installed
# !pip install azure-ai-projects azure-identity

# For additional functionality (optional):
# !pip install azure-ai-generative azure-cognitiveservices-speech

## Import Libraries

In [ ]:
import os
import json
import time
from datetime import datetime
from typing import List
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import FunctionTool

print("✅ Libraries imported successfully")

## 🔧 Configuration Setup

### Azure Credentials from App Registration
These values come from your **Azure App Registration** (Service Principal):

- **AZURE_CLIENT_ID**: Application (client) ID from App Registration
- **AZURE_TENANT_ID**: Directory (tenant) ID from your Azure AD
- **AZURE_CLIENT_SECRET**: Client secret generated in App Registration
- **project_endpoint**: Your Azure AI Foundry project endpoint URL

### Agent Configuration Options
- **AGENT_MODEL**: Choose the AI model (gpt-4o recommended for best performance)
- **AGENT_NAME**: Descriptive name for your agent instance
- **AGENT_INSTRUCTIONS**: System prompt that defines agent behavior

**⚠️ Security Note:** In production, store these credentials in Azure Key Vault or environment variables, never in code!

In [ ]:
# Azure Credentials from App Registration - Replace with your actual values
# Get these from: Azure Portal > Azure Active Directory > App registrations
os.environ["AZURE_CLIENT_ID"] = "your-application-client-id"        # Application (client) ID
os.environ["AZURE_TENANT_ID"] = "your-directory-tenant-id"          # Directory (tenant) ID 
os.environ["AZURE_CLIENT_SECRET"] = "your-client-secret-value"      # Client secret value

# Azure AI Project Configuration
# Format: https://<your-resource-name>.services.ai.azure.com/api/projects/<project-name>
project_endpoint = "https://your-resource.services.ai.azure.com/api/projects/your-project"

# Agent Configuration - Customize these for different use cases
AGENT_MODEL = "gpt-4o"  # Options: gpt-4o, gpt-4, gpt-35-turbo, gpt-4-turbo
AGENT_NAME = "demo-agent"  # Give your agent a descriptive name

# Agent Instructions - This is the "system prompt" that defines agent behavior
AGENT_INSTRUCTIONS = """You are a helpful AI assistant with access to several useful functions:
- Weather information retrieval
- Email sending capabilities  
- Current time/date information
- Mathematical calculations

Guidelines for responses:
- Always be clear about what actions you're taking
- Use functions when appropriate to fulfill user requests
- Provide detailed, helpful responses
- If you use multiple functions, explain the workflow
- Be professional but friendly in your communication"""

print("✅ Configuration set")
print(f"🤖 Agent Model: {AGENT_MODEL}")
print(f"📝 Agent Name: {AGENT_NAME}")
print(f"🔗 Project Endpoint: {project_endpoint.split('/')[2] if project_endpoint.startswith('https://') else 'Not configured'}")

## 🔧 Custom Function Definitions

### Understanding AI Agent Functions

Functions are the "tools" that AI agents can use to interact with external systems and perform tasks. Key concepts:

#### Function Requirements:
- **Type Hints**: Must have proper Python type annotations
- **Docstrings**: Clear descriptions for the AI to understand purpose
- **Error Handling**: Robust error handling for reliability
- **Return Types**: Consistent return format (usually strings)

#### How Function Calling Works:
1. **User makes request** → "Send weather to john@email.com"
2. **Agent analyzes request** → Determines it needs weather + email functions
3. **Agent calls functions** → `get_weather("New York")` then `send_email(...)`
4. **Functions execute** → Return results to agent
5. **Agent formulates response** → Uses function results in natural language

These functions demonstrate common enterprise use cases:

In [ ]:
def get_current_time() -> str:
    """Get current date and time"""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def get_weather(location: str) -> str:
    """Get weather information for a specific location
    
    Args:
        location: The city or location to get weather for
    
    Returns:
        str: Weather information for the location
    """
    weather_data = {
        "New York": "72°F, sunny with light clouds",
        "London": "65°F, rainy with occasional showers", 
        "Tokyo": "78°F, cloudy and humid",
        "Paris": "68°F, partly cloudy",
        "Sydney": "75°F, clear and sunny"
    }
    return weather_data.get(location, f"No weather data available for {location}")

def send_email(recipient: str, subject: str, body: str) -> str:
    """Send an email to a recipient
    
    Args:
        recipient: Email address of the recipient
        subject: Subject line of the email
        body: Body content of the email
    
    Returns:
        str: Success message
    """
    print(f"📧 Email to: {recipient}")
    print(f"📝 Subject: {subject}")
    print(f"📄 Body: {body}")
    return f"Email sent successfully to {recipient} with subject '{subject}'"

def calculate_numbers(numbers: List[float]) -> str:
    """Calculate sum of numbers
    
    Args:
        numbers: List of numbers to calculate sum
    
    Returns:
        str: The sum result
    """
    if not numbers:
        return "No numbers provided"
    total = sum(numbers)
    return f"Sum of {numbers} = {total}"

# List of functions for the agent
agent_functions = [get_current_time, get_weather, send_email, calculate_numbers]
print(f"✅ Functions ready: {[f.__name__ for f in agent_functions]}")

## 🤖 Create AI Agent

### Understanding the Agent Creation Process

This step creates the actual AI agent instance with:
- **Model Selection**: Determines intelligence level and capabilities
- **Function Registration**: Makes custom functions available to the agent
- **Instruction Set**: Defines agent personality and behavior
- **Resource Allocation**: Sets up compute and storage resources

### Model Options Explained:
- **gpt-4o**: Latest model, best performance, higher cost
- **gpt-4**: Reliable, excellent quality, moderate cost
- **gpt-4-turbo**: Faster processing, good for high-volume scenarios
- **gpt-35-turbo**: Cost-effective, suitable for simpler tasks

In [ ]:
def create_ai_agent():
    try:
        client = AIProjectClient(
            endpoint=project_endpoint,
            credential=DefaultAzureCredential()
        )
        
        functions = FunctionTool(functions=agent_functions)
        
        agent = client.agents.create_agent(
            model=AGENT_MODEL,
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=functions.definitions
        )
        
        print(f"✅ Agent created successfully!")
        print(f"🤖 Agent ID: {agent.id}")
        print(f"📝 Agent Name: {agent.name}")
        print(f"🔧 Model: {AGENT_MODEL}")
        return client, agent, functions
    except Exception as e:
        print(f"❌ Error: {e}")
        return None, None, None

client, agent, functions = create_ai_agent()

## Function Execution Helper

This handles the actual execution of functions when the agent calls them:

In [ ]:
def execute_function_call(function_name: str, function_args: dict) -> str:
    """Execute a function call with proper error handling"""
    try:
        if function_name == "get_current_time":
            return get_current_time()
        elif function_name == "get_weather":
            return get_weather(function_args.get("location", ""))
        elif function_name == "send_email":
            return send_email(
                function_args.get("recipient", ""),
                function_args.get("subject", ""),
                function_args.get("body", "")
            )
        elif function_name == "calculate_numbers":
            return calculate_numbers(function_args.get("numbers", []))
        else:
            return f"Unknown function: {function_name}"
    except Exception as e:
        return f"Error executing {function_name}: {str(e)}"

print("✅ Function execution helper ready")

## 🧵 Understanding Conversation Threads

### What are Threads?
A **thread** represents a complete conversation session between a user and the AI agent:

#### Thread Lifecycle:
```
1. Thread Created     → New conversation starts
2. Messages Added     → User and agent exchange messages
3. Context Maintained → Agent remembers previous messages
4. Functions Called   → Agent executes tasks as needed
5. Thread Persists    → Conversation history preserved
```

#### Thread vs Session vs Conversation:
- **Thread** = Technical term for conversation container
- **Session** = User-facing term for interaction period  
- **Conversation** = The actual back-and-forth dialogue

#### Memory and Context:
- Each thread maintains **conversation history**
- Agent can reference **previous messages** in same thread
- **Function call results** are remembered within thread
- **Separate threads** = Independent conversations (no shared memory)

#### Thread Management:
- **Automatic Creation**: New thread for each scenario
- **Resource Usage**: Threads consume storage (minimal cost)
- **Cleanup**: Typically auto-managed by Azure service
- **Persistence**: Threads survive agent restarts but are deleted when agent is deleted

### Scenario Runner with Function Call Handling

This function demonstrates the complete agent interaction lifecycle:
1. **Thread Creation** - Start new conversation
2. **Message Submission** - Send user request
3. **Run Processing** - Agent analyzes and plans
4. **Function Execution** - Execute required functions
5. **Response Generation** - Formulate final response

In [ ]:
def run_scenario_with_functions(user_message, scenario_name):
    """Run a scenario with proper function call handling"""
    if not agent:
        print("❌ No agent available")
        return
        
    print(f"🧪 {scenario_name}")
    print("-" * 60)
    print(f"USER: {user_message}")
    print()
    
    # Create conversation thread
    thread = client.agents.threads.create()
    
    # Send message
    message = client.agents.messages.create(
        thread_id=thread.id,
        role="user", 
        content=user_message
    )
    
    # Create run (not create_and_process to handle function calls manually)
    run = client.agents.runs.create(
        thread_id=thread.id,
        agent_id=agent.id
    )
    
    # Wait for completion and handle function calls
    max_attempts = 30
    attempts = 0
    
    while run.status in ["queued", "in_progress", "requires_action"] and attempts < max_attempts:
        time.sleep(2)
        run = client.agents.runs.get(thread_id=thread.id, run_id=run.id)
        print(f"📊 Status: {run.status}")
        attempts += 1
        
        # Handle function calls
        if run.status == "requires_action":
            print("🔧 Agent is calling functions...")
            
            required_actions = run.required_action
            if required_actions and hasattr(required_actions, 'submit_tool_outputs'):
                tool_calls = required_actions.submit_tool_outputs.tool_calls
                tool_outputs = []
                
                for tool_call in tool_calls:
                    function_name = tool_call.function.name
                    function_args = json.loads(tool_call.function.arguments)
                    
                    print(f"🔧 Calling: {function_name} with {function_args}")
                    
                    # Execute the function
                    result = execute_function_call(function_name, function_args)
                    print(f"✅ Result: {result}")
                    print()
                    
                    tool_outputs.append({
                        "tool_call_id": tool_call.id,
                        "output": str(result)
                    })
                
                # Submit results back to agent
                if tool_outputs:
                    run = client.agents.runs.submit_tool_outputs(
                        thread_id=thread.id,
                        run_id=run.id,
                        tool_outputs=tool_outputs
                    )
    
    if attempts >= max_attempts:
        print("⚠️ Run timed out")
        return
    
    # Get final response
    messages = client.agents.messages.list(thread_id=thread.id)
    for msg in reversed(list(messages)):
        if msg.role == "assistant":
            content = msg.content[0].text.value if msg.content else 'No content'
            print(f"🤖 AGENT: {content}")
            print()

print("✅ Scenario runner ready")

## 🧪 Scenario 1: Weather Report Email

### Learning Focus: Multi-step Function Chaining
This scenario demonstrates how AI agents can:
- **Chain multiple functions** together automatically
- **Extract parameters** from natural language requests
- **Combine data** from different sources into coherent output

**What happens internally:**
1. Agent parses: "Get weather for New York" → calls `get_weather("New York")`
2. Agent parses: "send it to john@example.com" → calls `send_email(...)`
3. Agent combines results into natural language response

**Watch for**: Function call sequence and parameter extraction

In [ ]:
run_scenario_with_functions(
    "Get the weather for New York and send it to john@example.com with subject 'Weather Update'",
    "SCENARIO 1: Weather Report Email"
)

## 🧪 Scenario 2: Time and Calculations

### Learning Focus: Parallel Function Execution
This scenario shows how agents can:
- **Handle multiple independent requests** in one message
- **Execute different types of functions** (utility vs computation)
- **Present organized results** from multiple operations

**What happens internally:**
1. Agent identifies two separate requests in one message
2. Calls `get_current_time()` and `calculate_numbers([10,20,30,40])`
3. Presents both results in structured format

**Watch for**: How agent organizes multiple function results

In [ ]:
run_scenario_with_functions(
    "What's the current time? Also calculate the sum of [10, 20, 30, 40]",
    "SCENARIO 2: Time and Calculations"
)

## 🧪 Scenario 3: Complex Multi-step Business Process

### Learning Focus: Advanced Workflow Automation
This scenario demonstrates enterprise-level capabilities:
- **Complex workflow execution** with multiple data sources
- **Data aggregation** from different functions
- **Business report generation** with automated delivery

**What happens internally:**
1. Gathers weather data: `get_weather("London")`
2. Performs calculations: `calculate_numbers([5,15,25])`
3. Aggregates results into report format
4. Delivers via email: `send_email("admin@company.com", ...)`

**Watch for**: How agent creates coherent business reports from multiple data sources

In [ ]:
run_scenario_with_functions(
    "Get weather for London, calculate sum of [5,15,25], and email results to admin@company.com with subject 'Daily Report'",
    "SCENARIO 3: Complex Multi-step Request"
)

## 🔬 Interactive Testing Lab

### Experiment with Your Own Scenarios

Use this section to test different combinations and explore agent capabilities:

#### Suggested Test Cases:
1. **Single Function Tests:**
   - `"What time is it?"`
   - `"What's the weather in Tokyo?"`
   - `"Calculate 100 + 200 + 300"`

2. **Multi-Function Combinations:**
   - `"Get time and weather for Paris"`
   - `"Calculate [1,2,3,4,5] and email result to test@example.com"`

3. **Complex Business Scenarios:**
   - `"Create a daily summary with current time, weather for Sydney, calculation of [50,75,100], and email it to manager@company.com"`

4. **Edge Cases:**
   - `"Get weather for a city that doesn't exist"`
   - `"Send email without specifying recipient"`
   - `"Calculate sum of empty list"`

#### What to Observe:
- 🔍 **Function selection logic** - Which functions does the agent choose?
- 📊 **Parameter extraction** - How does it parse your natural language?
- 🔄 **Error handling** - How does it handle invalid inputs?
- 📝 **Response formatting** - How does it present results?

In [ ]:
def test_custom_scenario(user_message):
    """Test with a custom message"""
    run_scenario_with_functions(user_message, "CUSTOM TEST")

# Example usage - uncomment to test:
# test_custom_scenario("What's the weather in Tokyo and what time is it?")
# test_custom_scenario("Calculate sum of [1,2,3,4,5] and send result to test@example.com")
# test_custom_scenario("Get current time and weather for Paris")

print("✅ Ready for custom testing!")
print("Use: test_custom_scenario('your message here')")

## 🧹 Resource Cleanup and Best Practices

### Understanding Azure AI Agent Costs

#### Cost Components:
1. **Agent Instance**: Compute resources for the AI model
2. **Function Execution**: Processing time for custom functions
3. **Message Processing**: Token usage for input/output
4. **Storage**: Thread and conversation history storage

#### Thread Lifecycle Management:
- **Creation**: New thread created for each conversation
- **Persistence**: Threads persist until agent deletion
- **Auto-cleanup**: Azure typically manages thread lifecycle automatically
- **Manual cleanup**: Not usually required, but agent deletion cleans all threads

#### Best Practices:
- ✅ **Always delete agents** when done with demos/testing
- ✅ **Monitor costs** in Azure Cost Management
- ✅ **Use lower-cost models** (gpt-35-turbo) for development
- ✅ **Set spending alerts** to avoid unexpected charges
- ✅ **Clean up regularly** in production environments

**⚠️ Important:** Always run cleanup when done to avoid unnecessary charges!

In [ ]:
def cleanup_agent():
    """Clean up the agent and associated resources"""
    if not agent or not client:
        print("⚠️  No agent or client to clean up")
        return
    
    try:
        print("🧹 Starting cleanup process...")
        print(f"🤖 Deleting agent: {agent.id} ({AGENT_NAME})")
        
        # Try different possible delete methods
        delete_methods = [
            ('delete', lambda: client.agents.delete(agent.id)),
            ('delete_agent', lambda: client.agents.delete_agent(agent.id)),
        ]
        
        deleted = False
        for method_name, delete_func in delete_methods:
            try:
                if hasattr(client.agents, method_name):
                    delete_func()
                    print(f"✅ Agent deleted successfully using {method_name}")
                    deleted = True
                    break
            except Exception as e:
                print(f"⚠️  Delete method '{method_name}' failed: {e}")
                continue
        
        if not deleted:
            print("⚠️  Automatic deletion failed - you may need to clean up manually")
            print("💡 Check the Azure AI Studio portal to delete the agent manually")
            print(f"🔗 Agent ID to delete: {agent.id}")
        else:
            print("🎉 Cleanup completed successfully!")
        
    except Exception as e:
        print(f"❌ Error during cleanup: {e}")
        print("💡 You may need to clean up manually in the Azure portal")
        print(f"🔗 Agent ID to delete: {agent.id}")

# Uncomment the line below to run cleanup:
# cleanup_agent()

print("✅ Cleanup function ready")
print("💡 Run cleanup_agent() when you're done testing")

## Demo Summary

This notebook demonstrates:

- ✅ **Custom Function Integration** - Adding weather, email, time, and calculation functions
- ✅ **Real-world Scenarios** - Practical use cases like weather reports and data processing
- ✅ **Multi-step Operations** - Agent combining multiple functions in one request
- ✅ **Function Call Handling** - Proper execution of agent function calls
- ✅ **Interactive Testing** - Easy way to test custom scenarios

### Key Components:
1. **Function Definitions** - Custom functions with proper type hints
2. **Function Executor** - Handles the actual execution of agent function calls
3. **Scenario Runner** - Manages the conversation flow and function call lifecycle
4. **Test Scenarios** - Demonstrates different capabilities
5. **Configurable Agent** - Easy to change model, name, and instructions

### Configuration Options:
- **AGENT_MODEL**: Choose from gpt-4o, gpt-4, gpt-35-turbo, etc.
- **AGENT_NAME**: Give your agent a custom name
- **AGENT_INSTRUCTIONS**: Customize the agent's behavior and personality

### Next Steps:
1. Replace mock functions with real APIs (weather services, email providers)
2. Add more complex business logic functions
3. Implement error handling and retry logic
4. Add logging and monitoring
5. Scale to handle multiple concurrent conversations

### 🧹 Important Cleanup:
**Always run `cleanup_agent()` when done** to delete the agent and avoid unnecessary Azure charges!

**Happy AI Agent building! 🚀**